# Importar precipitación mensual de AEMET a tabla postgres
AEMET pasa cada mes los datos de precipitación mensual (mm) de cada DH a la correspondiente CH en ficheros csv<br>
Cada fichero contiene los datos del mes (incompletos) y los de los dos meses anteriores<br>
Los datos del mes son incompletos y provisionales
Los datos del mes anterior completan el del fichero anterior para ese mes y pueden corregir datos suminstrados com anterioridad. Pueden ser incompetos<br>
Los datos del últimos con ya definitivos. Pueden corrigr los de ese mes facilitados en ficheros previos<br>

El proceso es el siguiente:
1. Se rellena la tupla fcsv con el nombre y dirección de los ficheros aemet a importar, por orden de datos más antiguos a más modernos
1. El script comprueba que los ficheros existen
1. El script pregunta los datos de la base de datos donde queremos importar (upsert) los datos de los ficheros csv
1. Se comrpueba que los datos de la base de datos permiten crear y cerrar una conexión 
1. Se crea la tabla tmp.aemet_2_oph_chs cuyas columnas son las columnas de los ficheros csv. Si cambian la estructura hay que cambiar la definición d ela tabla
1. Se borran los datos existentes en tmp.aemet_2_oph_chs
1. Se importan los datos del primver fichero csv
1. Se comprueba que todas las estaciones están dadas de alta en la tambla met.pexistencias. Si alguna no está se interrumpe el proceso y hay qye darla de alta
1. Se muestra en pantalla un resumen de los datos que se van a introducir
1. Se pide al usuario que acepte los datos a insertar en la tapla met.pmes
1. Si acepta se usert los los datos y se cntinua con el siguiente fichero csv; si no se acepta se interrumpe el proceso

In [23]:
import os.path
import traceback

import getpass
import psycopg2

## Ficheros csv a importar
Es fudamental colocarlos por orden en cuanto a los meses que contienen, de más antiguos a más modernos


In [7]:
fcsv = (
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_09_08_07_PrecMesSeg_utf8.csv',
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_10_09_08_PrecMesSeg_utf8.csv',
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_11_10_09_PrecMesSeg_utf8.csv',
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_12_11_10_PrecMesSeg_utf8.csv',
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2022_01_12_11_PrecMesSeg_utf8.csv',
    r'H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2022_02_01_12_PrecMesSeg_utf8.csv'
)

In [8]:
# Compruebo que existen
for f1 in fcsv:
    os.path.isfile(f1)
    print(f'{f1} existe')

H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_09_08_07_PrecMesSeg_utf8.csv existe
H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_10_09_08_PrecMesSeg_utf8.csv existe
H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_11_10_09_PrecMesSeg_utf8.csv existe
H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_12_11_10_PrecMesSeg_utf8.csv existe
H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2022_01_12_11_PrecMesSeg_utf8.csv existe
H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2022_02_01_12_PrecMesSeg_utf8.csv existe


In [17]:
# test connection
def db_con():
    db = getpass.getpass("Data base: ")
    user = getpass.getpass("user: ")
    password = getpass.getpass("password: ")
    return (db, user, password)

In [26]:
## test connecion
db, user, passw = db_con()
try:
    con = psycopg2.connect(database=db, user=user, password=passw)
except Exception as e:
    print(traceback.format_exc())
else:    
    con.close()
    print('connection opened and closed')

Data base:  ···
user:  ········
password:  ··


connection open and closed


In [29]:
# se crea una tabla con las mismas columnas que los ficheros csv
create_table = \
"""
create table if not exists tmp.aemet_2_oph_chs (
    indicativo varchar,
    año integer,
    mes integer,
    nombre varchar,
    altitud float,
    c_x float,
    c_y float,
    nom_prov varchar,
    pmes77 real
);
"""
try:
    con = psycopg2.connect(database=db, user=user, password=passw)
    cur = con.cursor()
    cur.execute(create_table)
    print('tabla disponible')
finally:
    con.close()

tabla disponible


In [43]:
stm_delete_rows = "delete from tmp.aemet_2_oph_chs;"
f1 = ''
test_indicativos = \
"""
select distinct t.indicativo , t2.indic 
from tmp.aemet_2_oph_chs t
    left join met.pexistencias t2 on(t.indicativo=t2.indic)
where t2.indic = NULL
order by t.indicativo , t2.indic;
"""
stm_view_2upsert = \
"""
select t.año , t.mes , count(*)
from tmp.aemet_2_oph_chs t
group by t.año , t.mes 
order by t.año , t.mes;
"""
stm_upsert = \
"""
insert into met.pmes (indic, fecha, prec) 
    select t.indicativo , (date_trunc('month', make_date(t.año , t.mes , 1)) + interval '1 month' - interval '1 day')::date as fecha , t.pmes77
    from tmp.aemet_2_oph_chs t
    order by t.indicativo , (date_trunc('month', make_date(t.año , t.mes , 1)) + interval '1 month' - interval '1 day')::date
on conflict on constraint pmes_pkey
do update set prec = excluded.prec;
"""
try:
    con = psycopg2.connect(database=db, user=user, password=passw)
    cur = con.cursor()
    for f1 in fcsv:
        print(f'{f1}')
        cur.execute(stm_delete_rows)
        stm_import = f"copy tmp.aemet_2_oph_chs from '{f1}' with CSV header delimiter ';' encoding 'UTF-8'"
        cur.execute(stm_import)
        print('importado')
        cur.execute(test_indicativos)
        n = 0
        print('estaciones no dadas de alta: ', end='')
        for row in cur.fetchall():
            print(row)
            n += 1
        if n == 0:
            print('todas las estaciones están dadas de alta')
        else:
            print(f'{n:d}')
            raise ValueError(r'Hay que dar de alta las nuevas estaciones')
        cur.execute(stm_view_2upsert)
        print('Resumen de contenidos del fichero csv')
        print('(Año, mes, número de datos)')
        for row in cur.fetchall():
            print(row)
        seguir = input("Continuar? (y/n): ")
        if seguir.strip().lower() != 'y':
            print('Ejecución interrumpida por el usuario')
            break
        # cur.execute(stm_upsert)
        print('Estaciones actualizadas')
except Exception as e:
    print(traceback.format_exc())
finally:
    con.close()

H:\IGME2020\_pz_manzano\20220324_informe_pz\aemet\2021_09_08_07_PrecMesSeg_utf8.csv
importado
estaciones no dadas de alta: todas las estaciones están dadas de alta
Resumen de contenidos del fichero csv
(Año, mes, número de datos)
(2021, 8, 109)
(2021, 9, 105)
(2021, 10, 49)


Continuar? (y/n):  n


Ejecución interrumpida por el usuario
